In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'/Users/jharna/work/self_learning/MLOps/kidney-disease-classification-deeplearning-pytorch'

In [3]:
%pwd

'/Users/jharna/work/self_learning/MLOps/kidney-disease-classification-deeplearning-pytorch'

[Pytorch VGG prepare model reference](https://www.analyticsvidhya.com/blog/2021/06/transfer-learning-using-vgg16-in-pytorch/)       
[Pytorch VGG model fine tuning reference 2](https://www.kaggle.com/code/sairampinjarla/image-classification-using-pytorch-vgg16-99-acc)

In [4]:
from cnnClassifierKidneyDisease.constants import *
from cnnClassifierKidneyDisease import logger
from cnnClassifierKidneyDisease.utils.common import read_yaml, create_directories
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: tuple
    params_learning_rate: float
    params_num_classes: int
    params_include_top: bool


In [6]:
class ConfigurationManager:
    def __init__(self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH) -> None:

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.image_size,
            params_learning_rate=self.params.learning_rate,
            params_num_classes=self.params.classes,
            params_include_top=self.params.include_top
        )

        return prepare_base_model_config

In [7]:
import torchvision
from torch import nn
from torchvision import datasets, transforms
import torchvision.models as models

In [8]:
config = ConfigurationManager()
prepare_base_model_config = config.get_prepare_base_model_config()


[2024-05-30 23:38:40,517: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-05-30 23:38:40,519: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-30 23:38:40,520: INFO: common: created directory at: artifacts]
[2024-05-30 23:38:40,521: INFO: common: created directory at: artifacts/prepare_base_model]


# Checking pytorch implementation for setting up customized model architecture

In [9]:
import torch

In [10]:
torch.backends.mps.is_available()

True

In [11]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
device

'mps'

In [12]:
model = models.vgg16()

In [13]:
list(model.children())

[Sequential(
   (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (1): ReLU(inplace=True)
   (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (3): ReLU(inplace=True)
   (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (6): ReLU(inplace=True)
   (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (8): ReLU(inplace=True)
   (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (11): ReLU(inplace=True)
   (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (13): ReLU(inplace=True)
   (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (15): ReLU(inplace=True)
   (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   (17): Conv2d(

In [14]:
nn.Sequential(*list(model.children()))

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [15]:
len(nn.Sequential(*list(model.children())))

3

In [16]:
nn.Sequential(*list(model.children()))[:-1]

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [17]:
# Removve the last sequential layer of VGG16 that contained the 3 linear layers
model = nn.Sequential(*list(model.children()))[-1]

In [18]:
for child in model.children():
    print(child)
    print("Hello")
    for grandchild in child.children():
        print(grandchild)
        print("Grandchild")

Linear(in_features=25088, out_features=4096, bias=True)
Hello
ReLU(inplace=True)
Hello
Dropout(p=0.5, inplace=False)
Hello
Linear(in_features=4096, out_features=4096, bias=True)
Hello
ReLU(inplace=True)
Hello
Dropout(p=0.5, inplace=False)
Hello
Linear(in_features=4096, out_features=1000, bias=True)
Hello


In [19]:
# Freeze the parameters of the pre-trained VGG16 layers
for parameters in model.parameters():
    parameters.requires_grad = False

In [20]:
# Define the custom classifier
classifier = nn.Sequential(
    nn.Flatten(),
    nn.Dropout(0.2),
    nn.Linear(512 * 7 * 7, 128),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(128, 2),
)

model = nn.Sequential(model, classifier).to(device)
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [19]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig) -> None:
        self.config = config

    def get_base_model(self):
        self.model = torchvision.models.vgg16()
        # Remove the last set of dense layers
        if self.config.params_include_top == False:
            self.model = nn.Sequential(*list(self.model.children())[:-1])
        
        self.save_model(model=self.model, path = self.config.base_model_path)

    @staticmethod
    def _prepare_full_model(model, classes, learning_rate):
        # Freeze the parameters of the pre-trained VGG16 layers
        for params in model.parameters():
                params.requires_grad = False

        device = 'mps' if torch.backends.mps.is_available() else 'cpu'

        # Define the custom classifier
        classifier = nn.Sequential(
            nn.Flatten(),
            # nn.Dropout(0.2),
            # nn.Linear(512 * 7 * 7, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(4096, classes),
        )

        full_model = nn.Sequential(model, classifier).to(device)
        loss_fn = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(full_model.parameters(), lr=learning_rate)

        return full_model

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_num_classes,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(model=self.full_model, path=self.config.updated_base_model_path)        
            

    @staticmethod
    def save_model(model, path: Path):
        torch.save(model, path)

In [21]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-05-30 23:45:21,290: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-05-30 23:45:21,298: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-30 23:45:21,304: INFO: common: created directory at: artifacts]
[2024-05-30 23:45:21,312: INFO: common: created directory at: artifacts/prepare_base_model]
